# Import statements

In [0]:
import gym
import tensorflow as tf
import numpy as np

## Implementation Details
1. Initialize the Parameters of Actor Network and Critic Network
2. For number of episodes:
  1. Sample Reward by taking a step from state s to s1
  2. The new loss minimization for vanilla policy gradients is log * advantage funtion
    1. Advantage Function= r+V(s1)-V(s)
  3. Traditional updates for the Q networks which uses the policy actions as it's target value

  Details taken from [link](https://towardsdatascience.com/understanding-actor-critic-methods-931b97b6df3f)

# A2C Vanilla Policy Critic

In [0]:
tf.compat.v1.disable_eager_execution()
input_state=tf.compat.v1.placeholder(tf.float32, shape=[None, state_size],name="input_state")
action_space=tf.compat.v1.placeholder(tf.int32, shape=[None, actions],name="action_space")
advantage_function=tf.compat.v1.placeholder(tf.float32, shape=[None,],name="advantage_funtion")
fc1=tf.keras.layers.Dense(10,activation='relu',name="fc1")(input_state)
fc2=tf.keras.layers.Dense(actions,activation='relu',name="fc2")(fc1)
fc3=tf.keras.layers.Dense(actions,name="fc3")(fc2)
action_output=tf.keras.layers.Softmax(name="action_output")(fc3)
neg_loss_prob=tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(logits = fc3, labels = action_space)
loss=tf.math.reduce_mean(advantage_function*neg_loss_prob)
training=tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(loss)